In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

Создадим датасет по фильмам. В качестве признаков будем использовать описание фильмов, а целевой функцией будет рейтинг фильма

In [3]:
n_samples = 1000

year = np.random.randint(2015, 2020, n_samples) #год выпуска фильма
runtime =  np.random.randint(90, 180, n_samples) #продолжительность фильма
viewers = np.random.choice(200, n_samples) + 10 #количество просмотров в миллионах
average_ticket_price = np.random.randint(5, 10, n_samples) #средняя стоимость просмотра в кинотеатре

box_office = viewers * average_ticket_price # пусть рейтинг зависит не от стоимости билета напрямую, а от суммарной прибыли за фильм

ratings = runtime // 5 + box_office // 25 + random.randint(-3,3) #пусть рейтинг фильма определяется по формуле, причем от года выпуска оценка зависеть не будет

data = pd.DataFrame({'year': year,'runtime': runtime,'viewers, mln': viewers, 'average_ticket_price, $': average_ticket_price, 'ratings':ratings})
data.head(5)

,year,runtime,"viewers, mln","average_ticket_price, $",ratings
0,2016,146,95,8,61
1,2016,118,133,9,72
2,2018,115,141,8,70
3,2016,92,99,6,43
4,2015,132,140,6,61


Попробуем построить модель на основе всех признаков

In [5]:
X = data[['year', 'runtime', 'viewers, mln',  'average_ticket_price, $']]
y = data['ratings']

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(data[['year', 'runtime', 'viewers, mln',  'average_ticket_price, $']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-0.07430835  0.19469335  0.27784987  4.24786801]
Bias: 122.27265824835075
Error: 2.513623669457259


Введем новый признак Box-office и заменим им в модели признаки средней стоимости и количества зрителей.

In [6]:
data['box_office'] = data['viewers, mln'] * data['average_ticket_price, $']
data.head(5)

,year,runtime,"viewers, mln","average_ticket_price, $",ratings,box_office
0,2016,146,95,8,61,760
1,2016,118,133,9,72,1197
2,2018,115,141,8,70,1128
3,2016,92,99,6,43,594
4,2015,132,140,6,61,840


In [7]:
X = data[['year', 'runtime', 'box_office']]
y = data['ratings']

reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(data[['year', 'runtime', 'box_office']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-2.45852884e-05  1.99801701e-01  3.99858590e-02]
Bias: 1.1946646525032207
Error: 0.31843708212324506


Отбросим первый признак с наименьшим весом

In [8]:
X = data[['runtime', 'box_office']]
y = data['ratings']
reg = LinearRegression().fit(X, y)

print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(data[['runtime', 'box_office']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.19980162 0.03998585]
Bias: 1.1450925586722036
Error: 0.3184366180124599
